\begin{center}
Gabe Morris
\end{center}

In [1]:
from scipy.interpolate import interp1d
from scipy.optimize import fsolve
import numpy as np

\pagebreak
\tableofcontents
\pagebreak

# Given
The preliminary design of a recovery heat exchanger for a Solar Turbines Centaur 50 is to be accomplished. The heat exchanger is to be used to heat water from the turbine exhaust for process use. Surface 8.0-3/8T or surface 9.29-0.737-SR is to be used. More information on the Centaur 50 may be found [here](https://s7d2.scene7.com/is/content/Caterpillar/CM20150703-52095-37893).

The requirements are:

| Property                    | Gas Side    | Water Side  |
|-----------------------------|-------------|-------------|
| $\dot{m}\,(\frac{lbm}{hr})$ | 151,410     | 36,000      |
| $T_{in}\,(^\circ F)$        | 910         | 70          |
| $T_{out}\,(^\circ F)$       | 400         | ?           |
| $P_{in}$                    | atmospheric | atmospheric |

The heat exchanger can have necessary width, but the gas inlet side must be such that the Reynolds number inside the finned-tube is about 1000. Water flows through the tubes, which are manifolded together in such a fashion that the water velocity is 3 ft/sec in order to ensure turbulent flow. The properties of the exhaust gas from the heat recovery are close to the properties of air at the same temperature.

# Find
Select the better surface based on economy of the operation by calculating (a) the heat exchanger width and volume for both surfaces, (b) the gas side pressure drop for both surfaces, and (c) the operating costs for both surfaces for 8760 hr/yr operation. Electricity costs $\frac{\$0.05}{kW\,hr}$ for usage and $\frac{\$9}{kW}$ for demand (per month). The fan is 75% efficient.

The surface selected must be clearly indicated in the report. A sketch of the selected configuration, with dimensions indicated should be included.

# Solution
## Properties
The first step is to acquire the properties of the air and water. The properties may be taken at the average temperature between the inlet and exiting temperatures.

### Air
The average temperature of the air is $655\,^\circ F$. The following properties may be obtained from Table B-2 in the text: $\rho=0.03554\,\frac{lbm}{ft^3}$, $c_p=0.25165\,\frac{Btu}{lbm\,^\circ F}$, $\mu=2.077\cdot10^{-5}\,\frac{lbm}{ft\,sec}$, $Pr=0.68775$.

### Water
For water, the exit temperature is not known, but it may be solved for using an iterative process. First, the rating must be solved for using the known values for the air properties.

In [2]:
# Input parameters of air
mdot_a = 151_410  # lbm/hr
Tin_a = 910  # F
Tout_a = 400  # F
cp_a = 0.25165  # btu/(lbm F)

# Getting the rating
q = mdot_a*cp_a*(Tin_a - Tout_a)
q  # Btu per hour

19432186.514999997

Now the exiting temperature may be solved using some python magic.

In [3]:
# Input parameters of water
mdot_w = 36_000  # lbm/hr
Tin_w = 70  # F

# We can get cp as a function of temperature using an interpolation function from scipy
# From the book,
T_values = [80, 90, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600]
cp_values = [0.998, 0.997, 0.998, 1, 1, 1.01, 1.03, 1.05, 1.08, 1.12, 1.19, 1.31, 1.51]

cp_w_lamb = interp1d(T_values, cp_values, fill_value='extrapolate')

Now `cp_w_lamb` may take in arguments at any temperature. This feature will be used later when accessing the database. For instance, the heat capacity should be between 1.08 and 1.12 for $410\,^\circ F$

In [4]:
float(cp_w_lamb(410))  # adding float() because it returns an array

1.088

Now `fsolve` may be used to iteratively find the outlet temperature.

In [5]:
# Getting the outlet temperature
def get_T(Tout_water):
    T_avg = (Tout_water + Tin_w)/2
    cp_water = float(cp_w_lamb(T_avg))
    return q - mdot_w*cp_water*(Tout_water - Tin_w)  # expression equal to zero

Tout_w = fsolve(get_T, np.array([500, ]))[0]
Tout_w  # F

588.2044624423871

With the average temperature of the water being $329.10223\,^\circ F$, the properties from Table B-2 are: $\rho=56.31\,\frac{lbm}{ft^3}$, $c_p=1.0416\,\frac{Btu}{lbm\,^\circ F}$, $\mu=0.114\cdot10^{-3}\,\frac{lbm}{ft\,sec}$, $Pr=1.087$.